<a href="https://colab.research.google.com/github/DUrayev/RepostCleanerBot/blob/main/Implementation_of_Spreadsheet_compression_techniques.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. STRUCTURAL ANCHORS FOR EFFICIENT LAYOUT UNDERSTANDING



This technique identifies heterogeneous rows/columns (structural anchors) and removes distant homogeneous areas that don't contribute to understanding spreadsheet layout

### Algorithm implementation

In [11]:
import pandas as pd
import numpy as np

def extract_structural_skeleton(df: pd.DataFrame, k: int = 1):
    """
    Extract structural skeleton by identifying structural anchors (boundary rows/columns)
    and keeping only k rows/columns around them, removing distant homogeneous regions.

    This implementation compares rows WITH EACH OTHER to identify structural changes,
    not the diversity within individual rows.
    """

    def rows_are_similar(row1_idx, row2_idx, similarity_threshold=0.8):
        """Check if two rows are structurally similar"""
        row1 = df.iloc[row1_idx].fillna('').astype(str)
        row2 = df.iloc[row2_idx].fillna('').astype(str)

        # Count how many positions have the same content
        matches = sum(1 for a, b in zip(row1, row2) if a == b)
        similarity = matches / len(row1)
        return similarity >= similarity_threshold

    def cols_are_similar(col1_idx, col2_idx, similarity_threshold=0.8):
        """Check if two columns are structurally similar"""
        col1 = df.iloc[:, col1_idx].fillna('').astype(str)
        col2 = df.iloc[:, col2_idx].fillna('').astype(str)

        # Count how many positions have the same content
        matches = sum(1 for a, b in zip(col1, col2) if a == b)
        similarity = matches / len(col1)
        return similarity >= similarity_threshold

    # Find structural anchor rows (rows where structure changes)
    row_anchors = set()
    for i in range(df.shape[0]):
        is_boundary = False

        # Check if this row is different from adjacent rows (structural boundary)
        if i == 0 or i == df.shape[0] - 1:
            is_boundary = True  # First and last rows are always anchors
        else:
            # Check if current row is significantly different from neighbors
            prev_similar = rows_are_similar(i-1, i)
            next_similar = rows_are_similar(i, i+1) if i+1 < df.shape[0] else False

            if not prev_similar and not next_similar:
                is_boundary = True

        if is_boundary:
            row_anchors.add(i)

    # Find structural anchor columns (columns where structure changes)
    col_anchors = set()
    for j in range(df.shape[1]):
        is_boundary = False

        # Check if this column is different from adjacent columns (structural boundary)
        if j == 0 or j == df.shape[1] - 1:
            is_boundary = True  # First and last columns are always anchors
        else:
            # Check if current column is significantly different from neighbors
            prev_similar = cols_are_similar(j-1, j)
            next_similar = cols_are_similar(j, j+1) if j+1 < df.shape[1] else False

            if not prev_similar or not next_similar:
                is_boundary = True

        if is_boundary:
            col_anchors.add(j)

    # Expand anchors by k (keep k rows/columns around each structural anchor)
    rows_to_keep = set()
    for r in row_anchors:
        rows_to_keep.update(range(max(0, r - k), min(df.shape[0], r + k + 1)))

    cols_to_keep = set()
    for c in col_anchors:
        cols_to_keep.update(range(max(0, c - k), min(df.shape[1], c + k + 1)))

    # Extract skeleton
    return df.iloc[sorted(rows_to_keep), sorted(cols_to_keep)]


### Example data

Sample DataFrame - realistic spreadsheet with truly homogeneous regions  
This demonstrates the power of structural anchors by having low-information density areas

In [12]:
data = {
    # Table 1: Sales Report (rows 0-3) - Distinct structure
    'A': ['Sales Report', 'Product', 'Laptop', 'Phone',
          # Homogeneous region (rows 4-11) - Repeated similar rows
          'Notes', 'Notes', 'Notes', 'Notes', 'Notes', 'Notes', 'Notes', 'Notes',
          # Table 2: Employee Data (rows 12-15) - Different structure
          'Employee List', 'Name', 'John Smith', 'Jane Doe',
          # Another homogeneous region (rows 16-23) - Different repeated pattern
          'Summary', 'Summary', 'Summary', 'Summary', 'Summary', 'Summary', 'Summary', 'Summary',
          # Table 3: Financial Data (rows 24-27) - Third distinct structure
          'Q4 Finances', 'Revenue', '$10000', '$15000'],

    'B': ['Q4 2024', 'Units', '150', '200',
          # Homogeneous region - Same pattern as column A
          'Details', 'Details', 'Details', 'Details', 'Details', 'Details', 'Details', 'Details',
          # Employee table - Different pattern
          'Department', 'Engineering', 'Engineering', 'Marketing',
          # Homogeneous region - Different repeated pattern
          'Info', 'Info', 'Info', 'Info', 'Info', 'Info', 'Info', 'Info',
          # Financial table - Third pattern
          'Target', 'Actual', '$12000', '$14000'],

    'C': ['Region', 'Price', '$1200', '$800',
          # Homogeneous region - Similar to A and B pattern
          'Extra', 'Extra', 'Extra', 'Extra', 'Extra', 'Extra', 'Extra', 'Extra',
          # Employee table - Matches employee pattern
          'Location', 'Seattle', 'Seattle', 'New York',
          # Homogeneous region - Matches summary pattern
          'More', 'More', 'More', 'More', 'More', 'More', 'More', 'More',
          # Financial - Matches financial pattern
          'Status', 'Goal Met', 'Yes', 'Yes'],

    'D': ['North', 'Count', '5', '8',
          # Homogeneous region - Similar repeated pattern
          'Misc', 'Misc', 'Misc', 'Misc', 'Misc', 'Misc', 'Misc', 'Misc',
          # Employee table - Matches employee pattern
          'Building', 'A', 'A', 'B',
          # Homogeneous region - Matches summary pattern
          'End', 'End', 'End', 'End', 'End', 'End', 'End', 'End',
          # Financial - Matches financial pattern
          'Quarter', 'Q4', 'Complete', 'Complete'],
}
df = pd.DataFrame(data)

This spreadsheet contains multiple tables separated by homogeneous regions:
- Rows 0-3: Sales table (different structure)
- Rows 4-11: Homogeneous region (repeated 'Notes/Details/Extra/Misc' pattern)
- Rows 12-15: Employee table (different structure)
- Rows 16-23: Homogeneous region (repeated 'Summary/Info/More/End' pattern)
- Rows 24-27: Financial table (different structure)

The algorithm will identify where structure changes and keep 'k' rows around each boundary.

Original DataFrame:

```
                A            B         C         D
0    Sales Report      Q4 2024    Region     North
1         Product        Units     Price     Count
2          Laptop          150     $1200         5
3           Phone          200      $800         8
4           Notes      Details     Extra      Misc
5           Notes      Details     Extra      Misc
6           Notes      Details     Extra      Misc
7           Notes      Details     Extra      Misc
8           Notes      Details     Extra      Misc
9           Notes      Details     Extra      Misc
10          Notes      Details     Extra      Misc
11          Notes      Details     Extra      Misc
12  Employee List   Department  Location  Building
13           Name  Engineering   Seattle         A
14     John Smith  Engineering   Seattle         A
15       Jane Doe    Marketing  New York         B
16        Summary         Info      More       End
17        Summary         Info      More       End
18        Summary         Info      More       End
19        Summary         Info      More       End
20        Summary         Info      More       End
21        Summary         Info      More       End
22        Summary         Info      More       End
23        Summary         Info      More       End
24    Q4 Finances       Target    Status   Quarter
25        Revenue       Actual  Goal Met        Q4
26         $10000       $12000       Yes  Complete
27         $15000       $14000       Yes  Complete
```



In [13]:
skeleton = extract_structural_skeleton(df, k=1)

### Display results with compression metrics

In [14]:
print("Original DataFrame:")
print(df)
print(f"\nStructural Anchor Extraction Results:")
print(f"  Original shape: {df.shape} ({df.shape[0] * df.shape[1]} cells)")
print(f"  Skeleton shape: {skeleton.shape} ({skeleton.shape[0] * skeleton.shape[1]} cells)")
compression_ratio = (df.shape[0] * df.shape[1]) / (skeleton.shape[0] * skeleton.shape[1])
print(f"  Compression ratio: {compression_ratio:.2f}x")
print(f"  Space saved: {((df.shape[0] * df.shape[1] - skeleton.shape[0] * skeleton.shape[1]) / (df.shape[0] * df.shape[1]) * 100):.1f}%")
print("\nExtracted Structural Skeleton:")
print(skeleton)

Original DataFrame:
                A            B         C         D
0    Sales Report      Q4 2024    Region     North
1         Product        Units     Price     Count
2          Laptop          150     $1200         5
3           Phone          200      $800         8
4           Notes      Details     Extra      Misc
5           Notes      Details     Extra      Misc
6           Notes      Details     Extra      Misc
7           Notes      Details     Extra      Misc
8           Notes      Details     Extra      Misc
9           Notes      Details     Extra      Misc
10          Notes      Details     Extra      Misc
11          Notes      Details     Extra      Misc
12  Employee List   Department  Location  Building
13           Name  Engineering   Seattle         A
14     John Smith  Engineering   Seattle         A
15       Jane Doe    Marketing  New York         B
16        Summary         Info      More       End
17        Summary         Info      More       End
18        S

The algorithm preserved table boundaries and structural anchors while removing
distant homogeneous regions that don't contribute to layout understanding.



```
Structural Anchor Extraction Results:
  Original shape: (28, 4) (112 cells)
  Skeleton shape: (16, 4) (64 cells)
  Compression ratio: 1.75x
  Space saved: 42.9%
```



### Patameter 'k' Demonstration

The parameter 'k' controls how many rows to keep around each structural anchor.
Lower k = more aggressive compression, Higher k = preserve more context

In [15]:
for k_val in [0, 1, 2]:
    test_skeleton = extract_structural_skeleton(df, k=k_val)
    compression = (df.shape[0] * df.shape[1]) / (test_skeleton.shape[0] * test_skeleton.shape[1])
    space_saved = ((df.shape[0] * df.shape[1] - test_skeleton.shape[0] * test_skeleton.shape[1]) / (df.shape[0] * df.shape[1]) * 100)
    print(f"k={k_val}: {df.shape} -> {test_skeleton.shape}, compression: {compression:.2f}x, space saved: {space_saved:.1f}%")

k=0: (28, 4) -> (12, 4), compression: 2.33x, space saved: 57.1%
k=1: (28, 4) -> (16, 4), compression: 1.75x, space saved: 42.9%
k=2: (28, 4) -> (20, 4), compression: 1.40x, space saved: 28.6%




```
k=0: (28, 4) -> (12, 4), compression: 2.33x, space saved: 57.1%
k=1: (28, 4) -> (16, 4), compression: 1.75x, space saved: 42.9%
k=2: (28, 4) -> (20, 4), compression: 1.40x, space saved: 28.6%
```



# 2. INVERTED INDEX TRANSLATION DEMONSTRATION

### Algorithm implementation




In [7]:
import pandas as pd
from collections import defaultdict
import itertools

def create_cell_address(row, col, df):
    """Convert row, col indices to Excel-style address like 'A1', 'B2', etc."""
    return f'{df.columns[col]}{row + 1}'

def merge_cell_ranges(addresses):
    """
    Merge consecutive cell addresses into ranges like A1:A4, B5:B7, etc.
    """
    if not addresses:
        return ""

    # Parse addresses into (column, row) tuples
    parsed = []
    for addr in addresses:
        col = addr[0]  # Assume single letter column for simplicity
        row = int(addr[1:])
        parsed.append((col, row, addr))

    # Sort by column then row
    parsed.sort()

    ranges = []
    current_range_start = None
    current_range_end = None
    current_col = None

    for col, row, addr in parsed:
        if current_col != col:
            # New column, finish previous range
            if current_range_start:
                if current_range_start == current_range_end:
                    ranges.append(current_range_start)
                else:
                    ranges.append(f"{current_range_start}:{current_range_end}")

            # Start new range
            current_col = col
            current_range_start = addr
            current_range_end = addr
        else:
            # Same column, check if consecutive
            prev_row = int(current_range_end[1:])
            if row == prev_row + 1:
                # Consecutive, extend range
                current_range_end = addr
            else:
                # Not consecutive, finish current range and start new one
                if current_range_start == current_range_end:
                    ranges.append(current_range_start)
                else:
                    ranges.append(f"{current_range_start}:{current_range_end}")
                current_range_start = addr
                current_range_end = addr

    # Finish last range
    if current_range_start:
        if current_range_start == current_range_end:
            ranges.append(current_range_start)
        else:
            ranges.append(f"{current_range_start}:{current_range_end}")

    return ','.join(ranges)

def invert_index_with_ranges(df: pd.DataFrame):
    """
    Create inverted index with proper range merging .
    Maps values to cell address ranges (e.g., "$100": "A1:A4", "$150": "B5,B7")
    """
    index = defaultdict(list)

    # Build value-to-addresses mapping
    for r, c in itertools.product(range(df.shape[0]), range(df.shape[1])):
        val = df.iat[r, c]
        if pd.notna(val) and str(val).strip() != '':
            addr = create_cell_address(r, c, df)
            index[str(val)].append(addr)

    # Merge addresses into ranges
    result = {}
    for val, addrs in index.items():
        result[val] = merge_cell_ranges(addrs)

    return result


Instead of traditional row-by-row serialization, this method creates a
value-to-address dictionary that:
- Maps identical cell values to their address ranges
- Eliminates redundant encoding of repeated values
- Merges consecutive addresses into ranges (e.g., A1:A4)
- Skips empty cells entirely

Create a realistic spreadsheet with repeated values that demonstrates the technique:

In [8]:
print("\n" + "-" * 60)
print("EXAMPLE: Financial Report with Repeated Values")
print("-" * 60)

# Create sample data with intentional repetitions to show the benefit
data = {
    'A': ['Quarter', 'Q1', 'Q1', 'Q1', 'Q2', 'Q2', 'Q2', '', 'Total', '$500K', '$500K'],
    'B': ['Product', 'Laptop', 'Laptop', 'Desktop', 'Laptop', 'Desktop', 'Desktop', '', 'Revenue', '$300K', '$200K'],
    'C': ['Revenue', '$100K', '$150K', '$100K', '$200K', '$100K', '$150K', '', 'Target', 'Met', 'Met'],
    'D': ['Status', 'Active', 'Active', 'Pending', 'Active', 'Pending', 'Active', '', 'Goal', 'Yes', 'Yes'],
    'E': ['Region', 'North', 'North', 'South', 'North', 'South', 'South', '', '', '', '']
}

df = pd.DataFrame(data)

print("ORIGINAL SPREADSHEET:")
print("Shape:", df.shape, f"({df.shape[0] * df.shape[1]} cells)")
print()
# Add row numbers for reference
df_display = df.copy()
df_display.index = [f'Row {i+1}' for i in range(len(df))]
print(df_display)


------------------------------------------------------------
EXAMPLE: Financial Report with Repeated Values
------------------------------------------------------------
ORIGINAL SPREADSHEET:
Shape: (11, 5) (55 cells)

              A        B        C        D       E
Row 1   Quarter  Product  Revenue   Status  Region
Row 2        Q1   Laptop    $100K   Active   North
Row 3        Q1   Laptop    $150K   Active   North
Row 4        Q1  Desktop    $100K  Pending   South
Row 5        Q2   Laptop    $200K   Active   North
Row 6        Q2  Desktop    $100K  Pending   South
Row 7        Q2  Desktop    $150K   Active   South
Row 8                                             
Row 9     Total  Revenue   Target     Goal        
Row 10    $500K    $300K      Met      Yes        
Row 11    $500K    $200K      Met      Yes        




```
ORIGINAL SPREADSHEET:
Shape: (11, 5) (55 cells)

              A        B        C        D       E
Row 1   Quarter  Product  Revenue   Status  Region
Row 2        Q1   Laptop    $100K   Active   North
Row 3        Q1   Laptop    $150K   Active   North
Row 4        Q1  Desktop    $100K  Pending   South
Row 5        Q2   Laptop    $200K   Active   North
Row 6        Q2  Desktop    $100K  Pending   South
Row 7        Q2  Desktop    $150K   Active   South
Row 8
Row 9     Total  Revenue   Target     Goal
Row 10    $500K    $300K      Met      Yes
Row 11    $500K    $200K      Met      Yes
```



### Traditional Encoding (Row-by-Row):

This would encode every cell individually:

In [9]:
print("\n" + "-" * 60)
print("TRADITIONAL ENCODING (Row-by-Row):")
print("-" * 60)
token_count = 0
for r in range(df.shape[0]):
    for c in range(df.shape[1]):
        val = df.iat[r, c]
        addr = create_cell_address(r, c, df)
        if pd.notna(val) and str(val).strip():
            print(f"  {addr}: '{val}'")
            token_count += 1
        elif str(val).strip() == '':
            print(f"  {addr}: ''")  # Empty cells still need to be encoded
            token_count += 1

print(f"\nTraditional method tokens: {token_count}")


------------------------------------------------------------
TRADITIONAL ENCODING (Row-by-Row):
------------------------------------------------------------
  A1: 'Quarter'
  B1: 'Product'
  C1: 'Revenue'
  D1: 'Status'
  E1: 'Region'
  A2: 'Q1'
  B2: 'Laptop'
  C2: '$100K'
  D2: 'Active'
  E2: 'North'
  A3: 'Q1'
  B3: 'Laptop'
  C3: '$150K'
  D3: 'Active'
  E3: 'North'
  A4: 'Q1'
  B4: 'Desktop'
  C4: '$100K'
  D4: 'Pending'
  E4: 'South'
  A5: 'Q2'
  B5: 'Laptop'
  C5: '$200K'
  D5: 'Active'
  E5: 'North'
  A6: 'Q2'
  B6: 'Desktop'
  C6: '$100K'
  D6: 'Pending'
  E6: 'South'
  A7: 'Q2'
  B7: 'Desktop'
  C7: '$150K'
  D7: 'Active'
  E7: 'South'
  A8: ''
  B8: ''
  C8: ''
  D8: ''
  E8: ''
  A9: 'Total'
  B9: 'Revenue'
  C9: 'Target'
  D9: 'Goal'
  E9: ''
  A10: '$500K'
  B10: '$300K'
  C10: 'Met'
  D10: 'Yes'
  E10: ''
  A11: '$500K'
  B11: '$200K'
  C11: 'Met'
  D11: 'Yes'
  E11: ''

Traditional method tokens: 55




```
Traditional method tokens: 55
```



### Inverted Index Translation:

In [10]:
print("\n" + "-" * 60)
print("INVERTED INDEX TRANSLATION:")
print("-" * 60)

# Apply inverted index
inverted = invert_index_with_ranges(df)

print("Value-to-Address Range Mapping:")
print("(Empty cells excluded, ranges merged where possible)")
print()

# Sort by value for better readability
for val in sorted(inverted.keys()):
    ranges = inverted[val]
    print(f"  '{val}': {ranges}")

print(f"\nInverted index entries: {len(inverted)}")
print(f"Token reduction: {token_count} -> {len(inverted)} ({(token_count - len(inverted))/token_count*100:.1f}% reduction)")


------------------------------------------------------------
INVERTED INDEX TRANSLATION:
------------------------------------------------------------
Value-to-Address Range Mapping:
(Empty cells excluded, ranges merged where possible)

  '$100K': C2,C4,C6
  '$150K': C3,C7
  '$200K': B11,C5
  '$300K': B10
  '$500K': A10:A11
  'Active': D2:D3,D5,D7
  'Desktop': B4,B6:B7
  'Goal': D9
  'Laptop': B2:B3,B5
  'Met': C10:C11
  'North': E2:E3,E5
  'Pending': D4,D6
  'Product': B1
  'Q1': A2:A4
  'Q2': A5:A7
  'Quarter': A1
  'Region': E1
  'Revenue': B9,C1
  'South': E4,E6:E7
  'Status': D1
  'Target': C9
  'Total': A9
  'Yes': D10:D11

Inverted index entries: 23
Token reduction: 55 -> 23 (58.2% reduction)


Token reduction: 55 -> 23 (58.2% reduction)

### Key Benifits Demonstrated:

1. REDUNDANCY ELIMINATION:
   - 'Q1' appears 3 times -> mapped once to 'A2:A4'
   - '$100K' appears 3 times -> mapped once to 'C2,C4,C6'
   - 'Active' appears 4 times -> mapped to ranges

2. RANGE MERGING:
   - Consecutive cells with same value become ranges
   - 'Q1' in A2,A3,A4 becomes 'A2:A4'
   - 'Q1' in A2,A3,A4 becomes 'A2:A4'
   - Non-consecutive cells stay separate: 'C2,C4,C6'

3. EMPTY CELL ELIMINATION:
   - Empty cells in column E and row 8 are completely skipped
   - No wasted tokens on encoding empty positions

4. LOSSLESS COMPRESSION:
   - All non-empty data is preserved
   - Exact cell positions are maintained
   - Can be perfectly reconstructed

   - 'Q1' in A2,A3,A4 becomes 'A2:A4'
   - Non-consecutive cells stay separate: 'C2,C4,C6'

This technique significantly reduces token usage while preserving
all structural and content information needed for LLM processing.

# 3. DATA FORMAT AGGREGATION

Technique Description:
Instead of encoding exact numerical values, this method:
- Groups cells by their data format/type (dates, currencies, percentages, etc.)
- Uses rule-based recognition for 9 predefined types:
  Year, Integer, Float, Percentage, Scientific notation, Date, Time, Currency, Email, Others
- Preserves semantic meaning while reducing token usage

### Algorithm implementation

In [18]:
import pandas as pd
import numpy as np
import re
from collections import defaultdict

def create_rule_based_recognizer():
    """
    Create rule-based recognizer for predefined data types:
    Year, Integer, Float, Percentage, Scientific notation, Date, Time, Currency, Email, and Others
    """

    def recognize_data_type(value):
        """Recognize data type based on cell value using rules"""
        if pd.isna(value) or str(value).strip() == '':
            return 'Empty'

        value_str = str(value).strip()

        # Year (4-digit number between reasonable range)
        if re.match(r'^(19|20|21)\d{2}$', value_str):
            return 'Year'

        # Email
        if re.match(r'^[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}$', value_str):
            return 'Email'

        # Percentage (ends with %)
        if re.match(r'^-?\d+\.?\d*%$', value_str):
            return 'Percentage'

        # Currency (starts with currency symbol)
        if re.match(r'^[\$£€¥₹]\s*-?\d{1,3}(,\d{3})*(\.\d{2})?$', value_str):
            return 'Currency'

        # Scientific notation
        if re.match(r'^-?\d+\.?\d*[eE][+-]?\d+$', value_str):
            return 'ScientificNotation'

        # Date (various formats)
        date_patterns = [
            r'^\d{4}[-/]\d{1,2}[-/]\d{1,2}$',  # YYYY-MM-DD or YYYY/MM/DD
            r'^\d{1,2}[-/]\d{1,2}[-/]\d{4}$',  # MM-DD-YYYY or MM/DD/YYYY
            r'^\d{1,2}[-/]\d{1,2}[-/]\d{2}$',  # MM-DD-YY or MM/DD/YY
        ]
        for pattern in date_patterns:
            if re.match(pattern, value_str):
                return 'Date'

        # Time (HH:MM or HH:MM:SS)
        if re.match(r'^\d{1,2}:\d{2}(:\d{2})?(\s*(AM|PM))?$', value_str, re.IGNORECASE):
            return 'Time'

        # Float (decimal number)
        if re.match(r'^-?\d+\.\d+$', value_str):
            return 'Float'

        # Integer (whole number)
        if re.match(r'^-?\d+$', value_str):
            return 'Integer'

        # Others (everything else)
        return 'Others'

    return recognize_data_type

def aggregate_by_data_format(df):
    """
    Aggregate cells by data format using rule-based recognition
    """
    recognizer = create_rule_based_recognizer()

    # Analyze each cell using rule-based recognition
    rule_groups = defaultdict(list)

    for i in range(df.shape[0]):
        for j in range(df.shape[1]):
            value = df.iat[i, j]
            cell_addr = f"{df.columns[j]}{i+1}"

            # Rule-based data type recognition
            data_type = recognizer(value)
            rule_groups[data_type].append({
                'address': cell_addr,
                'value': value,
                'type': data_type
            })

    return rule_groups

### Sample data

Create a sample DataFrame with various data types for demonstration

In [19]:
# Sample data with different data types
data = {
    'Category': ['Date', 'Date', 'Time', 'Time', 'Currency', 'Currency',
                'Percentage', 'Percentage', 'Integer', 'Integer', 'Float',
                'Float', 'Scientific', 'Year', 'Email', 'Phone', 'Text'],
    'Value': ['2024-01-15', '01/15/2024', '14:30:00', '2:30 PM', '$1,234.56',
              '£9,876.54', '75%', '12.5%', '42', '1000', '3.14159', '2.718',
              '6.022e23', '2024', 'user@example.com', '+1-555-123-4567', 'Hello World'],
    'Description': ['ISO Date Format', 'US Date Format', '24-hour Time',
                    '12-hour Time', 'US Dollar', 'British Pound', 'Percentage',
                    'Percentage (decimal)', 'Whole Number', 'Thousand',
                    'Pi (5 decimals)', 'e (3 decimals)', 'Avogadro Number',
                    'Year Format', 'Email Address', 'Phone Number', 'Text String']
}

df = pd.DataFrame(data)

print(f"DataFrame shape: {df.shape}")
print("\nSample data:")
print(df)

DataFrame shape: (17, 3)

Sample data:
      Category             Value           Description
0         Date        2024-01-15       ISO Date Format
1         Date        01/15/2024        US Date Format
2         Time          14:30:00          24-hour Time
3         Time           2:30 PM          12-hour Time
4     Currency         $1,234.56             US Dollar
5     Currency         £9,876.54         British Pound
6   Percentage               75%            Percentage
7   Percentage             12.5%  Percentage (decimal)
8      Integer                42          Whole Number
9      Integer              1000              Thousand
10       Float           3.14159       Pi (5 decimals)
11       Float             2.718        e (3 decimals)
12  Scientific          6.022e23       Avogadro Number
13        Year              2024           Year Format
14       Email  user@example.com         Email Address
15       Phone   +1-555-123-4567          Phone Number
16        Text       Hello

### Traditional Encoding (Every Value Individually)

In [20]:
print("\n" + "-" * 60)
print("TRADITIONAL ENCODING (Every Value Individually)")
print("-" * 60)

# Count original tokens (each cell encoded separately)
total_cells = 0
non_empty_cells = 0
for i in range(df.shape[0]):
    for j in range(df.shape[1]):
        total_cells += 1
        if pd.notna(df.iat[i, j]) and str(df.iat[i, j]).strip():
            non_empty_cells += 1
            cell_addr = f"{df.columns[j]}{i+1}"
            print(f"  {cell_addr}: '{df.iat[i, j]}'")

print(f"\nTraditional encoding tokens: {non_empty_cells}")


------------------------------------------------------------
TRADITIONAL ENCODING (Every Value Individually)
------------------------------------------------------------
  Category1: 'Date'
  Value1: '2024-01-15'
  Description1: 'ISO Date Format'
  Category2: 'Date'
  Value2: '01/15/2024'
  Description2: 'US Date Format'
  Category3: 'Time'
  Value3: '14:30:00'
  Description3: '24-hour Time'
  Category4: 'Time'
  Value4: '2:30 PM'
  Description4: '12-hour Time'
  Category5: 'Currency'
  Value5: '$1,234.56'
  Description5: 'US Dollar'
  Category6: 'Currency'
  Value6: '£9,876.54'
  Description6: 'British Pound'
  Category7: 'Percentage'
  Value7: '75%'
  Description7: 'Percentage'
  Category8: 'Percentage'
  Value8: '12.5%'
  Description8: 'Percentage (decimal)'
  Category9: 'Integer'
  Value9: '42'
  Description9: 'Whole Number'
  Category10: 'Integer'
  Value10: '1000'
  Description10: 'Thousand'
  Category11: 'Float'
  Value11: '3.14159'
  Description11: 'Pi (5 decimals)'
  Category

Traditional encoding tokens: 51

### Rule-based Data Type Aggregation

In [21]:
# Perform aggregation analysis
rule_groups = aggregate_by_data_format(df)

print("RULE-BASED DATA TYPE AGGREGATION:")
print("(Using predefined recognition patterns for 9 data types)")
for data_type, cells in sorted(rule_groups.items()):
    if data_type != 'Empty' and cells:
        addresses = [cell['address'] for cell in cells]
        sample_values = [str(cell['value']) for cell in cells[:3]]  # Show first 3 values
        sample_text = ', '.join(sample_values)
        if len(cells) > 3:
            sample_text += '...'
        print(f"   Type '{data_type}': {','.join(addresses)} (values: {sample_text})")

# Calculate compression
unique_types = len([group for group in rule_groups.values() if group and group[0]['type'] != 'Empty'])

compression_ratio = non_empty_cells / unique_types if unique_types > 0 else 1
space_saved = ((non_empty_cells - unique_types) / non_empty_cells * 100) if non_empty_cells > 0 else 0

print(f"\nCOMPRESSION RESULTS:")
print(f"   Original tokens: {non_empty_cells}")
print(f"   Aggregated types: {unique_types}")
print(f"   Compression ratio: {compression_ratio:.2f}x")
print(f"   Space saved: {space_saved:.1f}%")

RULE-BASED DATA TYPE AGGREGATION:
(Using predefined recognition patterns for 9 data types)
   Type 'Currency': Value5,Value6 (values: $1,234.56, £9,876.54)
   Type 'Date': Value1,Value2 (values: 2024-01-15, 01/15/2024)
   Type 'Email': Value15 (values: user@example.com)
   Type 'Float': Value11,Value12 (values: 3.14159, 2.718)
   Type 'Integer': Value9,Value10 (values: 42, 1000)
   Type 'Others': Category1,Description1,Category2,Description2,Category3,Description3,Category4,Description4,Category5,Description5,Category6,Description6,Category7,Description7,Category8,Description8,Category9,Description9,Category10,Description10,Category11,Description11,Category12,Description12,Category13,Description13,Category14,Description14,Category15,Description15,Category16,Value16,Description16,Category17,Value17,Description17 (values: Date, ISO Date Format, Date...)
   Type 'Percentage': Value7,Value8 (values: 75%, 12.5%)
   Type 'ScientificNotation': Value13 (values: 6.022e23)
   Type 'Time': Value3



```
COMPRESSION RESULTS:
   Original tokens: 51
   Aggregated types: 10
   Compression ratio: 5.10x
   Space saved: 80.4%
```



### Key Benifits Demonstrated:

1. SEMANTIC PRESERVATION:
   - Values '2024-01-15' and '01/15/2024' both recognized as 'Date'
   - Different currencies ($1234.56, £9876.54) grouped as 'Currency'
   - Various percentages (75%, 12.5%) unified as 'Percentage'

2. INTELLIGENT TYPE RECOGNITION:
   - Rule-based recognizer covers 9 common data types
   - Patterns detect semantic meaning (dates, currencies, emails, etc.)
   - Fallback to 'Others' for unrecognized patterns

3. STRUCTURE UNDERSTANDING:
   - Preserves data type semantics without exact values
   - Enables LLMs to understand column purposes and data patterns
   - Maintains enough information for structural analysis

4. TOKEN EFFICIENCY:
   - Dramatic reduction in tokens while preserving meaning
   - Groups cells by semantic type rather than exact content
   - Significant compression with minimal information loss

This technique enables LLMs to understand spreadsheet data types and structure without being overwhelmed by exact numerical values.